#3. Algoritmo QR

El método directo y el método de potencia permiten determinar un valor propio a la vez. El **algoritmo QR** o **método de descomposición QR**, en cambio, nos permite determinar todos los valores propios de una matriz.

<br>La parte esencial del algoritmo QR es la **descomposición QR** de una matriz. Existen varias formas de determinar dicha descomposición. Algunas de ellas son: el **proceso de ortogonalización de Gram-Schmidt**, mediante **rotaciones de Givens** y mediante **reflexiones de Householder**. En la implementación del algoritmo vamos a utilizar la función **linalg.qr** de Numpy para determinar la descomposición QR de una matriz $A$. Dicha función hace uso de las reflexiones de Householder para hallar la descomposición en cuestión. La intuición detrás del proceso es la siguiente: las reflexiones de Householder se aplican secuencialmente para "eliminar" los elementos por debajo de la diagonal de $A$, convirtiéndola en una matriz triangular superior $R$. Al mismo tiempo, las transformaciones acumuladas forman la matriz ortogonal $Q$. El costo computacional de éste método es similar al del proceso de ortogonalización de Gram-Schmidt (aunque es importante aclarar que se suele preferir el primero, pues el segundo suele "sufrir" más por los errores de redondeo). Nosotros, por simplicidad (para no perdernos en detalles ajenos al método como tal), vamos a utilizar la función mencionada para llevar a cabo la parte de la descomposición QR, sin profundizar en las reflexiones de Householder. Sin embargo, de ser absolutamente necesario, podríamos implementar fácilmente el conocido proceso de ortogonalización de Gram-Schmidt para llevar a cabo esta tarea.

<br>La estrategia general del algoritmo QR consiste en lo siguiente:

<br>1. Consideramos $A_0=A$.

<br>2. Hallamos la descomposición QR de $A_0$, para obtener $A_0=Q_0R_0$.

<br>3. Consideramos $A_1=R_0Q_0$.

<br>Lo que sigue es repetir el procedimiento antes dado, reemplazando el papel de $A_0$ con el de $A_1$, y continuar con las iteraciones. De este modo, tendremos en general que:

<br>-- Si $n$ es par, $A_n=Q_nR_n$. Ésto es: determinamos la descomposición QR de $A_n$.

<br>-- Si $n$ es impar, $A_n=R_{n-1}Q_{n-1}$. Ésto es: consideramos la matriz a la cual le hallaremos la descomposición QR en el siguiente paso.

<br>Las matrices $A_n$ y $A_{n+1}$ son **semejantes**, lo que nos asegura que tienen los mismos valores propios (pero no los mismos vectores propios). Ésto nos asegura, por así decirlo, que durante las iteraciones no perdemos de vista los valores propios de la matriz A. Además, la matriz $A_n$ se aproxima a una matriz triangular superior, en la cual podemos encontrar los valores propios en la diagonal principal. Ésta aproximación de $A_n$ a una matriz triangular superior no se justifica en ningún libro de la bibliografía. Está demostrada, por supuesto, pero la cuestión parece ser un poco difícil, por lo que tampoco será abordada aquí. Bastará decir que, intuitivamente, el algoritmo QR "reorganiza" progresivamente la matriz $A$ para reflejar mejor su estructura espectral, reduciendo las entradas fuera de la diagonal.

<br>De acuerdo con Hoffman, la convergencia del algoritmo está asegurada (por lo general), aunque podría ser algo lenta. Para acelerar la convergencia, se suelen llevar a cabo dos modificaciones:

<br>-- **Transformar la matriz de partida:** Generalmente lo que se hace es transformar la matriz $A$ en otra matriz "casi" triangular. Formalmente, se utilizan **transformaciones de Householder** para convertir la matriz $A$ en una **matriz de Hessenberg**.

<br>-- **Shifting:** Mediante lo que Hoffman llama "shifting" de los valores propios de la matriz de Hessenberg de la cual se parte, a medida que se ejecuta el algoritmo, se puede mejorar significativamente la convergencia del método. Hay varias formas de llevar a cabo dicho "shifting", de acuerdo a ciertas consideraciones.



##Implementación del algoritmo QR

In [2]:

import numpy as np #importamos la librería numpy que utilizaremos para trabajar con matrices

#La siguiente función toma como parámetros una Lista de tamaño nxn, un número máximo de iteraciones opcional,
#que por defecto será 100, y una tolerancia numérica, también opcional
def QRDecomposition(A, maxIteraciones = 1000, tol = 1e-10):
    A_n = np.array(A) #Convertimos la lista en un array de numpy para operar con el fácilmente

    for i in range(maxIteraciones): #Comenzamos a iterar el método

        #Cuando i es par:
        if i%2 == 0:
            Q_n,R_n = np.linalg.qr(A_n) #Descomposición QR de la matriz usando la función integrada de numpy

        #Cuando i es impar
        if i%2 == 1:
            A_n = R_n@Q_n

    """Sabemos que en este método los valores que convergen a los valores propios están ubicados
     sobre la diagonal de la matriz A_n, entonces vamos a recorrer esa diagonal y guardarlos en una lista"""

    valoresPropios = [] #Creamos la lista
    for i in range(A_n.shape[0]): #.shape[0] es para contar el número de filas, el cual será igual al numero de valores en la diagonal
                                  #dado que la matriz es cuadrada
        valoresPropios.append(A_n[i,i]) #Ingresamos los valores con índice [i,i] en la diagonal

    return valoresPropios

In [3]:
import numpy as np

# Declaramos los vectores correspondientes para los casos de prueba.
vector = np.array([1, 1]) #Dimensión 2.
vector1 = np.array([1, 2, 0]) #Dimensión 3.
vector2 = np.array([1, 1, 1, 1]) #Dimensión 4.

#Casos de prueba: declaramos un diccionario con las matrices de prueba.

test = {"A": np.array([[2, 1], [3, 4]]),
       "B": np.array([[3, 2], [3, 4]]),
       "C": np.array([[2, 3], [1, 4]]),
       "D": np.array([[1, 1, 2], [2, 1, 1],[1, 1, 3]]),
       "E": np.array([[1, 1, 2], [2, 1, 3],[1, 1, 1]]),
       "F": np.array([[2, 1, 2], [1, 1, 3],[1, 1, 1]]),
       "G": np.array([[1, 1, 1, 2], [2, 1, 1, 1],[3, 2, 1, 2],[2, 1, 1, 4]]),
        "H": np.array([[1, 2, 1, 2], [2, 1, 1, 1],[3, 2, 1, 2],[2, 1, 1, 4]])}

for nombre, matriz in test.items(): #Creamos un ciclo que recorre el diccionario y va probando cada caso
  print(f"Los valores propios de la matriz {nombre} son:")
  print(QRDecomposition(matriz)) #Utilizamos la funcion QRDecomposition e imprimimos cada valor propio
  print()

Los valores propios de la matriz A son:
[4.999999999999999, 0.9999999999999997]

Los valores propios de la matriz B son:
[5.999999999999998, 1.0000000000000002]

Los valores propios de la matriz C son:
[5.0, 1.0]

Los valores propios de la matriz D son:
[4.507018644092973, 0.7781238377368096, -0.28514248182978574]

Los valores propios de la matriz E son:
[4.048917339522307, -0.692021471630096, -0.3568958678922091]

Los valores propios de la matriz F son:
[4.124885419764577, -0.7615571818318913, 0.6366717620673171]

Los valores propios de la matriz G son:
[6.634534463633592, 1.5085633449433231, -0.7356415384387974, -0.4074562701381241]

Los valores propios de la matriz H son:
[6.827262250104039, 1.7281159082896407, -1.0879349236625606, -0.4674432347311208]



##Conclusiones

<br>--**Ventajas y desventajas del método directo:** Aquí nos referimos al método directo que utiliza a su vez el método de Newton. Entre las ventajas del método directo encontramos: bajo ciertas condiciones, el método es preciso y converge rápidamente. Es una opción razonable para matrices pequeñas. Algunas desventajas: para matrices grandes, el coste computacional aumenta significativamente (por la cantidad de operaciones que es necesario llevar a cabo; entre ellas, por ejemplo, determinar el polinomio característico). Por razones similares al punto anterior, el método puede ser numéricamente inestable. Como vimos, si hay valores propios repetidos, el método converge más lento, y puede incluso fallar.

<br>--**Ventajas y desventajas del método de las potencias:** Entre las ventajas del método de las potencias encontramos: es fácil y eficiente de implementar, dado que solo necesita operaciones básicas y el almacenamiento de pocos elementos (una matriz y un vector en cada iteración). Es especialmente útil para matrices grandes y **dispersas**. El hecho de encontrar el vector propio asociado al valor propio dominante también es una ventaja, pues los otros métodos solamente determinan valores propios.  Algunas desventajas: solamente encuentra el valor propio dominante. Requiere de matrices bien condicionadas y con espectros sencillos (se mencionó, por ejemplo, que si el valor propio dominante no está claramente separado de los demás, la convergencia del método podría ser lenta o inestable).  

<br>--**Ventajas y desventajas del algoritmo QR:** Entre las ventajas del algoritmo QR encontramos: determina todos los valores propios de la matriz. El método es estable y funciona bien incluso para matrices mal condicionadas. Algunas desventajas: es computacionalmente costoso. Converge lentamente. Las operaciones que se llevan a cabo durante el método (en particular, la factorización QR) requieren un consumo de memoria considerable.